In [8]:
import pandas as pd

from IPython.display import display
from geopy.geocoders import Nominatim

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [9]:
data = pd.read_csv( '../Datasets/data-filter.csv' )

In [10]:
# new date format
data['year'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y' )
data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%U' )


In [11]:
# create is_waterfront row
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'no' if x == 0 else 'yes' )

In [12]:
# create level for each quartile of price
quartile = list( data['price'].quantile([0.25, 0.5, 0.75]) )

data['price_level'] = data['price'].apply( lambda x: 'level_one'   if x <= quartile[0] else
                                                     'level_two'   if ( x > quartile[0] ) & ( x <= quartile[1] ) else
                                                     'level_three' if ( x > quartile[1] ) & ( x <= quartile[2] ) else
                                                     'level_four' )

In [13]:
data[['price', 'price_level']].sample(20)

,price,price_level
12395,615000.00,level_three
1339,435000.00,level_two
12195,685000.00,level_four
6251,408000.00,level_two
16400,578000.00,level_three
19208,359000.00,level_two
7035,3800000.00,level_four
19984,249900.00,level_one
3904,975000.00,level_four
5778,417500.00,level_two


In [14]:
# create a condition status
condition = list( data['condition'].unique() )
condition.sort()

data['status_condition'] = data['condition'].apply( lambda x: 'bad' if x <= condition[1] else
                                                    'regular' if x == condition[2] else
                                                    'good' if x == condition[3] else 'excelent' )

In [15]:
data['renovated'] = data['yr_renovated'].apply( lambda x: 'no' if x == 0 else 'yes' )

In [20]:
data['price/m2'] = data['price']/( data['sqft_lot']*0.0929 )

In [21]:
data

,id,date,price,bedrooms,bathrooms,sqft_lot,floors,waterfront,condition,yr_built,...,zipcode,lat,long,year,year_week,is_waterfront,price_level,status_condition,renovated,price/m2
0,7129300520,20141013T000000,221900.00,3,1.00,5650,1.00,0,3,1955,...,98178,47.51,-122.26,2014,2014-41,no,level_one,regular,no,422.76
1,6414100192,20141209T000000,538000.00,3,2.25,7242,2.00,0,3,1951,...,98125,47.72,-122.32,2014,2014-49,no,level_three,regular,yes,799.66
2,5631500400,20150225T000000,180000.00,2,1.00,10000,1.00,0,3,1933,...,98028,47.74,-122.23,2015,2015-08,no,level_one,regular,no,193.76
3,2487200875,20141209T000000,604000.00,4,3.00,5000,1.00,0,5,1965,...,98136,47.52,-122.39,2014,2014-49,no,level_three,excelent,no,1300.32
4,1954400510,20150218T000000,510000.00,3,2.00,8080,1.00,0,3,1987,...,98074,47.62,-122.05,2015,2015-07,no,level_three,regular,no,679.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21607,263000018,20140521T000000,360000.00,3,2.50,1131,3.00,0,3,2009,...,98103,47.70,-122.35,2014,2014-20,no,level_two,regular,no,3426.29
21608,6600060120,20150223T000000,400000.00,4,2.50,5813,2.00,0,3,2014,...,98146,47.51,-122.36,2015,2015-08,no,level_two,regular,no,740.70
21609,1523300141,20140623T000000,402101.00,2,0.75,1350,2.00,0,3,2009,...,98144,47.59,-122.30,2014,2014-25,no,level_two,regular,no,3206.16
21610,291310100,20150116T000000,400000.00,3,2.50,2388,2.00,0,3,2004,...,98027,47.53,-122.07,2015,2015-02,no,level_two,regular,no,1803.06


In [8]:
data.loc[data['condition'] == 5, ['condition','status']].sample(10)

,condition,status
18317,5,excelent
9144,5,excelent
5456,5,excelent
5105,5,excelent
7608,5,excelent
10426,5,excelent
19525,5,excelent
12413,5,excelent
4474,5,excelent
294,5,excelent


In [10]:
# create adress data

# create empty rows:
data['road'] = 'NA'
data['house_number'] = 'NA'
data['neighbourhood'] = 'NA'
data['postcode'] = 'NA'
data['city'] = 'NA'
data['state'] = 'NA'

# initialize API Nominatim
geolocator = Nominatim( user_agent='geopiRequest' )


for i in range( len( data ) ):
    print( 'Loop : {}/{}'.format( i, len(data) ) )
    # create query
    query = str( data.loc[i,'lat'] ) + ',' + str( data.loc[i,'long'] )
    
    # API request
    response = geolocator.reverse( query, timeout=None )
    
    # populate data:
    if 'road' in response.raw['address']:
        data.loc[i, 'road'] = response.raw['address']['road']
        
    if 'house_number' in response.raw['address']:
        data.loc[i, 'house_number'] = response.raw['address']['house_number']
    
    if 'neighbourhood' in response.raw['address']:
        data.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']
    
    if 'postcode' in response.raw['address']:
        data.loc[i, 'postcode'] = response.raw['address']['postcode']
    
    if 'city' in response.raw['address']:
        data.loc[i, 'city'] = response.raw['address']['city']
    
    if 'state' in response.raw['address']:
        data.loc[i, 'state'] = response.raw['address']['state']
    


Loop : 0/21612
Loop : 1/21612
Loop : 2/21612
Loop : 3/21612
Loop : 4/21612
Loop : 5/21612
Loop : 6/21612
Loop : 7/21612
Loop : 8/21612
Loop : 9/21612
Loop : 10/21612
Loop : 11/21612
Loop : 12/21612
Loop : 13/21612
Loop : 14/21612
Loop : 15/21612
Loop : 16/21612
Loop : 17/21612
Loop : 18/21612
Loop : 19/21612
Loop : 20/21612
Loop : 21/21612
Loop : 22/21612
Loop : 23/21612
Loop : 24/21612
Loop : 25/21612
Loop : 26/21612
Loop : 27/21612
Loop : 28/21612
Loop : 29/21612
Loop : 30/21612
Loop : 31/21612
Loop : 32/21612
Loop : 33/21612
Loop : 34/21612
Loop : 35/21612
Loop : 36/21612
Loop : 37/21612
Loop : 38/21612
Loop : 39/21612
Loop : 40/21612
Loop : 41/21612
Loop : 42/21612
Loop : 43/21612
Loop : 44/21612
Loop : 45/21612
Loop : 46/21612
Loop : 47/21612
Loop : 48/21612
Loop : 49/21612
Loop : 50/21612
Loop : 51/21612
Loop : 52/21612
Loop : 53/21612
Loop : 54/21612
Loop : 55/21612
Loop : 56/21612
Loop : 57/21612
Loop : 58/21612
Loop : 59/21612
Loop : 60/21612
Loop : 61/21612
Loop : 62/21612
Lo

Loop : 489/21612
Loop : 490/21612
Loop : 491/21612
Loop : 492/21612
Loop : 493/21612
Loop : 494/21612
Loop : 495/21612
Loop : 496/21612
Loop : 497/21612
Loop : 498/21612
Loop : 499/21612
Loop : 500/21612
Loop : 501/21612
Loop : 502/21612
Loop : 503/21612
Loop : 504/21612
Loop : 505/21612
Loop : 506/21612
Loop : 507/21612
Loop : 508/21612
Loop : 509/21612
Loop : 510/21612
Loop : 511/21612
Loop : 512/21612
Loop : 513/21612
Loop : 514/21612
Loop : 515/21612
Loop : 516/21612
Loop : 517/21612
Loop : 518/21612
Loop : 519/21612
Loop : 520/21612
Loop : 521/21612
Loop : 522/21612
Loop : 523/21612
Loop : 524/21612
Loop : 525/21612
Loop : 526/21612
Loop : 527/21612
Loop : 528/21612
Loop : 529/21612
Loop : 530/21612
Loop : 531/21612
Loop : 532/21612
Loop : 533/21612
Loop : 534/21612
Loop : 535/21612
Loop : 536/21612
Loop : 537/21612
Loop : 538/21612
Loop : 539/21612
Loop : 540/21612
Loop : 541/21612
Loop : 542/21612
Loop : 543/21612
Loop : 544/21612
Loop : 545/21612
Loop : 546/21612
Loop : 547/216

Loop : 971/21612
Loop : 972/21612
Loop : 973/21612
Loop : 974/21612
Loop : 975/21612
Loop : 976/21612
Loop : 977/21612
Loop : 978/21612
Loop : 979/21612
Loop : 980/21612
Loop : 981/21612
Loop : 982/21612
Loop : 983/21612
Loop : 984/21612
Loop : 985/21612
Loop : 986/21612
Loop : 987/21612
Loop : 988/21612
Loop : 989/21612
Loop : 990/21612
Loop : 991/21612
Loop : 992/21612
Loop : 993/21612
Loop : 994/21612
Loop : 995/21612
Loop : 996/21612
Loop : 997/21612
Loop : 998/21612
Loop : 999/21612
Loop : 1000/21612
Loop : 1001/21612
Loop : 1002/21612
Loop : 1003/21612
Loop : 1004/21612
Loop : 1005/21612
Loop : 1006/21612
Loop : 1007/21612
Loop : 1008/21612
Loop : 1009/21612
Loop : 1010/21612
Loop : 1011/21612
Loop : 1012/21612
Loop : 1013/21612
Loop : 1014/21612
Loop : 1015/21612
Loop : 1016/21612
Loop : 1017/21612
Loop : 1018/21612
Loop : 1019/21612
Loop : 1020/21612
Loop : 1021/21612
Loop : 1022/21612
Loop : 1023/21612
Loop : 1024/21612
Loop : 1025/21612
Loop : 1026/21612
Loop : 1027/21612
Loo

Loop : 1428/21612
Loop : 1429/21612
Loop : 1430/21612
Loop : 1431/21612
Loop : 1432/21612
Loop : 1433/21612
Loop : 1434/21612
Loop : 1435/21612
Loop : 1436/21612
Loop : 1437/21612
Loop : 1438/21612
Loop : 1439/21612
Loop : 1440/21612
Loop : 1441/21612
Loop : 1442/21612
Loop : 1443/21612
Loop : 1444/21612
Loop : 1445/21612
Loop : 1446/21612
Loop : 1447/21612
Loop : 1448/21612
Loop : 1449/21612
Loop : 1450/21612
Loop : 1451/21612
Loop : 1452/21612
Loop : 1453/21612
Loop : 1454/21612
Loop : 1455/21612
Loop : 1456/21612
Loop : 1457/21612
Loop : 1458/21612
Loop : 1459/21612
Loop : 1460/21612
Loop : 1461/21612
Loop : 1462/21612
Loop : 1463/21612
Loop : 1464/21612
Loop : 1465/21612
Loop : 1466/21612
Loop : 1467/21612
Loop : 1468/21612
Loop : 1469/21612
Loop : 1470/21612
Loop : 1471/21612
Loop : 1472/21612
Loop : 1473/21612
Loop : 1474/21612
Loop : 1475/21612
Loop : 1476/21612
Loop : 1477/21612
Loop : 1478/21612
Loop : 1479/21612
Loop : 1480/21612
Loop : 1481/21612
Loop : 1482/21612
Loop : 148

Loop : 1884/21612
Loop : 1885/21612
Loop : 1886/21612
Loop : 1887/21612
Loop : 1888/21612
Loop : 1889/21612
Loop : 1890/21612
Loop : 1891/21612
Loop : 1892/21612
Loop : 1893/21612
Loop : 1894/21612
Loop : 1895/21612
Loop : 1896/21612
Loop : 1897/21612
Loop : 1898/21612
Loop : 1899/21612
Loop : 1900/21612
Loop : 1901/21612
Loop : 1902/21612
Loop : 1903/21612
Loop : 1904/21612
Loop : 1905/21612
Loop : 1906/21612
Loop : 1907/21612
Loop : 1908/21612
Loop : 1909/21612
Loop : 1910/21612
Loop : 1911/21612
Loop : 1912/21612
Loop : 1913/21612
Loop : 1914/21612
Loop : 1915/21612
Loop : 1916/21612
Loop : 1917/21612
Loop : 1918/21612
Loop : 1919/21612
Loop : 1920/21612
Loop : 1921/21612
Loop : 1922/21612
Loop : 1923/21612
Loop : 1924/21612
Loop : 1925/21612
Loop : 1926/21612
Loop : 1927/21612
Loop : 1928/21612
Loop : 1929/21612
Loop : 1930/21612
Loop : 1931/21612
Loop : 1932/21612
Loop : 1933/21612
Loop : 1934/21612
Loop : 1935/21612
Loop : 1936/21612
Loop : 1937/21612
Loop : 1938/21612
Loop : 193

...

Loop : 5076/21612
Loop : 5077/21612
Loop : 5078/21612
Loop : 5079/21612
Loop : 5080/21612
Loop : 5081/21612
Loop : 5082/21612
Loop : 5083/21612
Loop : 5084/21612
Loop : 5085/21612
Loop : 5086/21612
Loop : 5087/21612
Loop : 5088/21612
Loop : 5089/21612
Loop : 5090/21612
Loop : 5091/21612
Loop : 5092/21612
Loop : 5093/21612
Loop : 5094/21612
Loop : 5095/21612
Loop : 5096/21612
Loop : 5097/21612
Loop : 5098/21612
Loop : 5099/21612
Loop : 5100/21612
Loop : 5101/21612
Loop : 5102/21612
Loop : 5103/21612
Loop : 5104/21612
Loop : 5105/21612
Loop : 5106/21612
Loop : 5107/21612
Loop : 5108/21612
Loop : 5109/21612
Loop : 5110/21612
Loop : 5111/21612
Loop : 5112/21612
Loop : 5113/21612
Loop : 5114/21612
Loop : 5115/21612
Loop : 5116/21612
Loop : 5117/21612
Loop : 5118/21612
Loop : 5119/21612
Loop : 5120/21612
Loop : 5121/21612
Loop : 5122/21612
Loop : 5123/21612
Loop : 5124/21612
Loop : 5125/21612
Loop : 5126/21612
Loop : 5127/21612
Loop : 5128/21612
Loop : 5129/21612
Loop : 5130/21612
Loop : 513

Loop : 5532/21612
Loop : 5533/21612
Loop : 5534/21612
Loop : 5535/21612
Loop : 5536/21612
Loop : 5537/21612
Loop : 5538/21612
Loop : 5539/21612
Loop : 5540/21612
Loop : 5541/21612
Loop : 5542/21612
Loop : 5543/21612
Loop : 5544/21612
Loop : 5545/21612
Loop : 5546/21612
Loop : 5547/21612
Loop : 5548/21612
Loop : 5549/21612
Loop : 5550/21612
Loop : 5551/21612
Loop : 5552/21612
Loop : 5553/21612
Loop : 5554/21612
Loop : 5555/21612
Loop : 5556/21612
Loop : 5557/21612
Loop : 5558/21612
Loop : 5559/21612
Loop : 5560/21612
Loop : 5561/21612
Loop : 5562/21612
Loop : 5563/21612
Loop : 5564/21612
Loop : 5565/21612
Loop : 5566/21612
Loop : 5567/21612
Loop : 5568/21612
Loop : 5569/21612
Loop : 5570/21612
Loop : 5571/21612
Loop : 5572/21612
Loop : 5573/21612
Loop : 5574/21612
Loop : 5575/21612
Loop : 5576/21612
Loop : 5577/21612
Loop : 5578/21612
Loop : 5579/21612
Loop : 5580/21612
Loop : 5581/21612
Loop : 5582/21612
Loop : 5583/21612
Loop : 5584/21612
Loop : 5585/21612
Loop : 5586/21612
Loop : 558

Loop : 5988/21612
Loop : 5989/21612
Loop : 5990/21612
Loop : 5991/21612
Loop : 5992/21612
Loop : 5993/21612
Loop : 5994/21612
Loop : 5995/21612
Loop : 5996/21612
Loop : 5997/21612
Loop : 5998/21612
Loop : 5999/21612
Loop : 6000/21612
Loop : 6001/21612
Loop : 6002/21612
Loop : 6003/21612
Loop : 6004/21612
Loop : 6005/21612
Loop : 6006/21612
Loop : 6007/21612
Loop : 6008/21612
Loop : 6009/21612
Loop : 6010/21612
Loop : 6011/21612
Loop : 6012/21612
Loop : 6013/21612
Loop : 6014/21612
Loop : 6015/21612
Loop : 6016/21612
Loop : 6017/21612
Loop : 6018/21612
Loop : 6019/21612
Loop : 6020/21612
Loop : 6021/21612
Loop : 6022/21612
Loop : 6023/21612
Loop : 6024/21612
Loop : 6025/21612
Loop : 6026/21612
Loop : 6027/21612
Loop : 6028/21612
Loop : 6029/21612
Loop : 6030/21612
Loop : 6031/21612
Loop : 6032/21612
Loop : 6033/21612
Loop : 6034/21612
Loop : 6035/21612
Loop : 6036/21612
Loop : 6037/21612
Loop : 6038/21612
Loop : 6039/21612
Loop : 6040/21612
Loop : 6041/21612
Loop : 6042/21612
Loop : 604

Loop : 6444/21612
Loop : 6445/21612
Loop : 6446/21612
Loop : 6447/21612
Loop : 6448/21612
Loop : 6449/21612
Loop : 6450/21612
Loop : 6451/21612
Loop : 6452/21612
Loop : 6453/21612
Loop : 6454/21612
Loop : 6455/21612
Loop : 6456/21612
Loop : 6457/21612
Loop : 6458/21612
Loop : 6459/21612
Loop : 6460/21612
Loop : 6461/21612
Loop : 6462/21612
Loop : 6463/21612
Loop : 6464/21612
Loop : 6465/21612
Loop : 6466/21612
Loop : 6467/21612
Loop : 6468/21612
Loop : 6469/21612
Loop : 6470/21612
Loop : 6471/21612
Loop : 6472/21612
Loop : 6473/21612
Loop : 6474/21612
Loop : 6475/21612
Loop : 6476/21612
Loop : 6477/21612
Loop : 6478/21612
Loop : 6479/21612
Loop : 6480/21612
Loop : 6481/21612
Loop : 6482/21612
Loop : 6483/21612
Loop : 6484/21612
Loop : 6485/21612
Loop : 6486/21612
Loop : 6487/21612
Loop : 6488/21612
Loop : 6489/21612
Loop : 6490/21612
Loop : 6491/21612
Loop : 6492/21612
Loop : 6493/21612
Loop : 6494/21612
Loop : 6495/21612
Loop : 6496/21612
Loop : 6497/21612
Loop : 6498/21612
Loop : 649

GeocoderServiceError: HTTP Error 502: Bad Gateway

In [3]:
# initialize API Nominatim
geolocator = Nominatim( user_agent='geopiRequest' )

# make query
query = '-23.963421, -46.317879'


# API request:
response = geolocator.reverse( query )

response.raw


{'place_id': 186684601,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 366666717,
 'lat': '-23.96342741341243',
 'lon': '-46.31788338673272',
 'display_name': 'Rua Nabuco de Araújo, Embaré, Santos, Região Imediata de Santos, Região Metropolitana da Baixada Santista, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 11025-003, Brasil',
 'address': {'road': 'Rua Nabuco de Araújo',
  'suburb': 'Embaré',
  'city_district': 'Santos',
  'city': 'Santos',
  'municipality': 'Região Imediata de Santos',
  'county': 'Região Metropolitana da Baixada Santista',
  'state_district': 'Região Geográfica Intermediária de São Paulo',
  'state': 'São Paulo',
  'region': 'Região Sudeste',
  'postcode': '11025-003',
  'country': 'Brasil',
  'country_code': 'br'},
 'boundingbox': ['-23.9665225', '-23.9632088', '-46.318203', '-46.3133374']}

In [86]:
data[['lat', 'long']].head()


,lat,long
0,47.51,-122.26
1,47.72,-122.32
2,47.74,-122.23
3,47.52,-122.39
4,47.62,-122.05


In [14]:
display( data )

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,yr_built,yr_renovated,zipcode,lat,long,year,year_week,is_waterfront
0,7129300520,2014-10-13,221900.0,3,1.00,1.0,0,3,1955,0,98178,47.5112,-122.257,2014,2014-41,no
1,6414100192,2014-12-09,538000.0,3,2.25,2.0,0,3,1951,1991,98125,47.7210,-122.319,2014,2014-49,no
2,5631500400,2015-02-25,180000.0,2,1.00,1.0,0,3,1933,0,98028,47.7379,-122.233,2015,2015-08,no
3,2487200875,2014-12-09,604000.0,4,3.00,1.0,0,5,1965,0,98136,47.5208,-122.393,2014,2014-49,no
4,1954400510,2015-02-18,510000.0,3,2.00,1.0,0,3,1987,0,98074,47.6168,-122.045,2015,2015-07,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21607,263000018,2014-05-21,360000.0,3,2.50,3.0,0,3,2009,0,98103,47.6993,-122.346,2014,2014-20,no
21608,6600060120,2015-02-23,400000.0,4,2.50,2.0,0,3,2014,0,98146,47.5107,-122.362,2015,2015-08,no
21609,1523300141,2014-06-23,402101.0,2,0.75,2.0,0,3,2009,0,98144,47.5944,-122.299,2014,2014-25,no
21610,291310100,2015-01-16,400000.0,3,2.50,2.0,0,3,2004,0,98027,47.5345,-122.069,2015,2015-02,no


In [9]:
data[['id','is_waterfront']].groupby( 'is_waterfront' ).count()

,id
is_waterfront,
no,21449
yes,163
